# Projeto Integrador 4 - UNIVESP
## Análise de dados para previsão de diabetes com Machine Learning
Alunos: Marcus Vinicius Camargo Pereira | Wallace Cosme da Silva
---

In [ ]:
# ===============================================================
# 1. Instalar biblioteca necessária
# ===============================================================

!pip install gspread

In [ ]:
# ===============================================================
# 2. Importar bibliotecas necessárias e autenticar o Google
# ===============================================================

from google.colab import auth
import gspread
import pandas as pd
import numpy as np
import gspread_dataframe as gd
from sklearn.impute import SimpleImputer

# Autenticação do usuário
auth.authenticate_user()
from google.auth import default
creds, _ = default()

# Autoriza o gspread
gc = gspread.authorize(creds)

print("Autenticação concluída com sucesso.")

In [ ]:
# ===============================================================
# 3. Carregamento dos dados do Google Sheets (Treinamento e previsão)
# ===============================================================

NOME_DA_PLANILHA = "Base de dados PI"
ABA_DADOS_ORIGINAIS = "Dados"
ABA_NOVOS_DADOS = "Respostas ao formulário 1"

df_treino = pd.DataFrame()
df_para_prever = pd.DataFrame()
lista_dfs_novos = []

try:
    print(f"Abrindo a planilha: '{NOME_DA_PLANILHA}'...")
    planilha = gc.open(NOME_DA_PLANILHA)

    # 1. Carrega os dados originais para TREINO
    print(f"Lendo a aba de TREINO: '{ABA_DADOS_ORIGINAIS}'...")
    aba_originais = planilha.worksheet(ABA_DADOS_ORIGINAIS)
    df_treino = pd.DataFrame(aba_originais.get_all_records())
    print(f"Encontrados {len(df_treino)} registros para TREINO.")

    # 2. Carrega os novos dados (do Formulário) para PREVISÃO
    try:
        print(f"Lendo a aba de PREVISÃO: '{ABA_NOVOS_DADOS}'...")
        aba_novos = planilha.worksheet(ABA_NOVOS_DADOS)
        df_para_prever = pd.DataFrame(aba_novos.get_all_records())
        lista_dfs_novos.append(df_para_prever)

        if 'Carimbo de data/hora' in df_para_prever.columns:
            df_para_prever = df_para_prever.drop(columns=['Carimbo de data/hora'])
        elif 'Timestamp' in df_para_prever.columns:
            df_para_prever = df_para_prever.drop(columns=['Timestamp'])

        print(f"Encontrados {len(df_para_prever)} novos registros para PREVISÃO.")

    except gspread.exceptions.WorksheetNotFound:
        print(f"Aba '{ABA_NOVOS_DADOS}' não encontrada ou vazia. Nenhum dado para prever.")

except Exception as e:
    print(f"\nERRO: Não foi possível abrir a planilha. Verifique o nome '{NOME_DA_PLANILHA}' e se você deu permissão.")
    print(e)

In [ ]:
# ===============================================================
# 4. Tradução dos nomes para português
# ===============================================================

mapa_nomes = {
    'Pregnancies': 'Gravidez',
    'Glucose': 'Glicose',
    'BloodPressure': 'PressaoArterial',
    'SkinThickness': 'EspessuraPele',
    'Insulin': 'Insulina',
    'BMI': 'IMC',
    'DiabetesPedigreeFunction': 'FuncaoGeneticaDiabetes',
    'Age': 'Idade',
    'Outcome': 'Resultado'
}

df_treino = df_treino.rename(columns=mapa_nomes)
print("Colunas de TREINO traduzidas.")


if not df_para_prever.empty:
    df_para_prever = df_para_prever.rename(columns=mapa_nomes)
    print("Colunas de PREVISÃO traduzidas.")

In [ ]:
# ===============================================================
# 5. Limpeza de dados
# ===============================================================

cols_zero_na = ['Glicose', 'PressaoArterial', 'EspessuraPele', 'Insulina', 'IMC']
df = df_treino.copy() # Usar 'df' como variável temporária para o treino

# 1. Lidar com strings vazias (caso haja algum erro)
df[cols_zero_na] = df[cols_zero_na].replace('', np.nan)
df[cols_zero_na] = df[cols_zero_na].replace(' ', np.nan)

# 2. Forçar colunas para tipo numérico
for col in cols_zero_na:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 3. Substituir zeros
print("Substituindo valores 0 por NaN nos dados de TREINO...")
df[cols_zero_na] = df[cols_zero_na].replace(0, np.nan)

df_treino = df.copy() # Atualiza o df_treino principal
print("\nValores ausentes (TREINO):")
print(df_treino.isnull().sum())

In [ ]:
# ===============================================================
# 6. Imputar dados ausentes
# ===============================================================

from sklearn.impute import SimpleImputer

print("Preparando o Imputer (calculando medianas) nos dados de TREINO...")
imputer = SimpleImputer(strategy='median')

# Separar as características do alvo para imputação
# O imputador deve aprender e imputar apenas colunas de características
X_for_imputation = df_treino.drop('Resultado', axis=1)
y_original = df_treino['Resultado']

# O Imputer é treinado (ajustado) e aplicado (transformado) apenas nas características
X_imputado = pd.DataFrame(imputer.fit_transform(X_for_imputation), columns=X_for_imputation.columns)

# Recombina as características imputadas com a variável alvo original
df_imputado = X_imputado.copy()
df_imputado['Resultado'] = y_original

# Preservar tipos de dados originais (assuming Resultado is int)
df_imputado['Resultado'] = df_imputado['Resultado'].astype(int)

print("Valores ausentes após imputação (TREINO):")
print(df_imputado.isnull().sum())
df_treino = df_imputado # Atualiza o df_treino

In [ ]:
# ===============================================================
# 7. Análise Exploratória
# ===============================================================

import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

print(df_treino.describe().T)

sns.countplot(x='Resultado', data=df_treino, palette='viridis')
plt.title('Distribuição de Casos de Diabetes (Dados de Treino)')
plt.show()

plt.figure(figsize=(12, 8))
sns.heatmap(df_treino.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap de Correlação (Dados de Treino)')
plt.show()

In [ ]:
# ===============================================================
# 8. Separação dos dados de treino e previsão
# ===============================================================

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Divisão em features (X) e target (y) - A PARTIR DO DF_TREINO
X = df_treino.drop('Resultado', axis=1)
y = df_treino['Resultado']

# 2. Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

print(f"Formato de X_train: {X_train.shape}")
print(f"Formato de X_test: {X_test.shape}")

# 3. Padronização (Scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nDados de treino padronizados com sucesso.")

In [ ]:
# ===============================================================
# 9. Treinamento do modelo
# ===============================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle

# --- 1. Random Forest (Modelo Escolhido) ---
rf = RandomForestClassifier(random_state=42, n_estimators=200)

print("Treinando o modelo Random Forest...")
# Usamos os dados NÃO padronizados para o RF
rf.fit(X_train, y_train)
print("Modelo treinado com sucesso.")

# --- 2. Regressão Logística (Para Comparação) ---
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# ===============================================================
# 10. Avaliação do modelo
# ===============================================================

# Previsões no conjunto de TESTE (do df_treino)
y_pred_rf = rf.predict(X_test)
y_proba_rf_test = rf.predict_proba(X_test)[:, 1] # Probabilidade da classe 1 (Diabetes)

print("--- Avaliação do Random Forest (nos 25% de dados de teste) ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_rf) * 100:.2f}%")
print(classification_report(y_test, y_pred_rf))

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Previsto: Não', 'Previsto: Sim'],
            yticklabels=['Real: Não', 'Real: Sim'])
plt.title('Matriz de Confusão - Dados de Teste')
plt.show()

In [ ]:
# ===============================================================
# 11. Importância das variáveis
# ===============================================================

# Criar um DataFrame com a importância das features
importancias = pd.DataFrame({
    'Variavel': X_train.columns,
    'Importancia': rf.feature_importances_
})
importancias = importancias.sort_values(by='Importancia', ascending=False)
print(importancias)

sns.barplot(x='Importancia', y='Variavel', data=importancias, palette='mako')
plt.title('Importância das Variáveis (Random Forest)')
plt.show()

In [ ]:
# ===============================================================
# 12. Limpeza dos novos dados
# ===============================================================

if not df_para_prever.empty:
    print("Limpando os novos dados do formulário...")

    # --------------------------------------------------------------------------------------------------
    # 1. TRADUÇÃO DO FORMULÁRIO (DE INGLÊS PARA PORTUGUÊS)
    # Assumimos que o formulário usa os nomes padrão do dataset Pima (em Inglês).
    # --------------------------------------------------------------------------------------------------
    mapa_traducao = {
        'Pregnancies': 'Gravidez',
        'Glucose': 'Glicose',
        'BloodPressure': 'PressaoArterial',
        'SkinThickness': 'EspessuraPele',
        'Insulin': 'Insulina',
        'BMI': 'IMC',
        'DiabetesPedigreeFunction': 'FuncaoGeneticaDiabetes',
        'Age': 'Idade',
    }

    # Aplica a tradução nos dados do formulário
    df_para_prever = df_para_prever.rename(columns=mapa_traducao)
    print("Colunas do formulário traduzidas de Inglês para Português.")


    cols_features = X_train.columns # Pega a lista ['Gravidez', 'Glicose', ...]
    cols_zero_na = ['Glicose', 'PressaoArterial', 'EspessuraPele', 'Insulina', 'IMC']

    try:
        # Agora deve funcionar, pois os nomes foram traduzidos para PT-BR
        X_novos = df_para_prever[cols_features].copy()
    except KeyError as e:
        print("\n--- ERRO CRÍTICO (KeyError) NA SELEÇÃO FINAL ---")
        print("O nome da coluna ainda não está no padrão do modelo, mesmo após a tradução.")
        print(f"COLUNAS ESPERADAS (Modelo): {list(cols_features)}")
        print(f"COLUNAS ATUAIS (Formulário APÓS TRADUÇÃO): {list(df_para_prever.columns)}")
        raise e

    # 1. Limpeza de strings vazias e conversão para numérico
    X_novos[cols_zero_na] = X_novos[cols_zero_na].replace('', np.nan)
    X_novos[cols_zero_na] = X_novos[cols_zero_na].replace(' ', np.nan)
    for col in cols_zero_na:
        X_novos[col] = pd.to_numeric(X_novos[col], errors='coerce')

    # 2. Substituir zeros por NaN (regra de limpeza do Pima)
    X_novos[cols_zero_na] = X_novos[cols_zero_na].replace(0, np.nan)

    # 3. Imputação (Etapa 6) - Usando o imputer JÁ TREINADO
    print("Aplicando imputação (medianas do treino) nos novos dados...")
    X_novos_imputado = pd.DataFrame(imputer.transform(X_novos), columns=X_novos.columns)

    mapa_reverso = {v: k for k, v in mapa_traducao.items()}
    X_novos_imputado_en = X_novos_imputado.rename(columns=mapa_reverso)

    print("Limpeza e preparação dos novos dados concluída.")

else:
    print("Nenhum dado novo do formulário para limpar ou prever.")

In [ ]:
# ===============================================================
# 13. Previsão dos novos dados
# ===============================================================

if not df_para_prever.empty:
    print("Executando previsões nos novos pacientes...")

    # Usar o modelo 'rf' treinado nos dados de treino
    # para prever nos novos dados imputados
    novas_previsoes = rf.predict(X_novos_imputado)
    novas_probabilidades = rf.predict_proba(X_novos_imputado)[:, 1]

    # Adicionar as previsões de volta ao DataFrame original do formulário
    df_para_prever['Resultado_Previsto'] = novas_previsoes
    df_para_prever['Probabilidade_Prevista'] = novas_probabilidades

    print("Previsões concluídas e adicionadas ao DataFrame.")
    print(df_para_prever[['Gravidez', 'Glicose', 'IMC', 'Idade', 'Resultado_Previsto', 'Probabilidade_Prevista']].head())

else:
    print("Nenhuma previsão a ser feita.")

In [ ]:
# ===============================================================
# 14. Salvar o modelo e os resultados para o Power BI
# ===============================================================

import pickle
from google.colab import files

print("Iniciando preparação dos dados unificados...")

# 1. Preparar o df_treino (Dados Históricos)
df_final_treino = df_treino.copy()
# Renomeia 'Resultado' para um nome final
df_final_treino.rename(columns={'Resultado': 'Resultado_Final'}, inplace=True)
# Adiciona uma coluna para identificar a origem do dado
df_final_treino['Tipo_Dado'] = 'Histórico'
df_final_treino['Probabilidade'] = np.nan # Histórico não tem probabilidade da previsão

# 2. Preparar o df_para_prever (Novos Pacientes)
df_final_previsao = pd.DataFrame()
if not df_para_prever.empty:
    df_final_previsao = df_para_prever.copy()
    # Renomeia colunas para bater com o df_final_treino
    df_final_previsao.rename(columns={
        'Resultado_Previsto': 'Resultado_Final',
        'Probabilidade_Prevista': 'Probabilidade'
    }, inplace=True)
    df_final_previsao['Tipo_Dado'] = 'Previsão'

# 3. Juntar (Concatenar) os dois DataFrames
# Assegurar que ambos tenham as mesmas colunas antes de concatenar
colunas_finais = ['Gravidez', 'Glicose', 'PressaoArterial', 'EspessuraPele',
                  'Insulina', 'IMC', 'FuncaoGeneticaDiabetes', 'Idade',
                  'Resultado_Final', 'Tipo_Dado', 'Probabilidade']

# Reordenar colunas (e preencher com NaN se faltar alguma)
df_final_treino = df_final_treino.reindex(columns=colunas_finais)
if not df_final_previsao.empty:
    df_final_previsao = df_final_previsao.reindex(columns=colunas_finais)

df_publicar = pd.concat([df_final_treino, df_final_previsao], ignore_index=True)

# 4. Limpar o DataFrame para o Google Sheets (gspread não gosta de NaT/NaN)
df_publicar = df_publicar.fillna("") # Substitui NaN por string vazia

print(f"Dados unificados criados. Total de linhas: {len(df_publicar)}")

# 5. Salvar o DataFrame de volta na planilha
NOME_ABA_POWER_BI = "PowerBI_Unificado"

try:
    print(f"Tentando acessar a aba '{NOME_ABA_POWER_BI}'...")
    # Tenta encontrar a aba
    worksheet = planilha.worksheet(NOME_ABA_POWER_BI)
    print("Aba encontrada. Limpando dados antigos...")
    worksheet.clear() # Limpa tudo antes de escrever

except gspread.exceptions.WorksheetNotFound:
    print("Aba não encontrada. Criando uma nova...")
    # Cria a aba se ela não existir
    worksheet = planilha.add_worksheet(title=NOME_ABA_POWER_BI, rows=10, cols=10) # Tamanho inicial pequeno

# 6. Escrever os dados na aba
print(f"Escrevendo {len(df_publicar)} linhas na aba '{NOME_ABA_POWER_BI}'...")
gd.set_with_dataframe(worksheet, df_publicar, include_index=False, include_column_header=True, resize=True)

print("\n--- SUCESSO! ---")
print(f"Seu dashboard unificado está pronto na aba '{NOME_ABA_POWER_BI}' da sua planilha.")
print("Agora, conecte o Power BI diretamente nesta aba.")

# 7. (Opcional) Download dos modelos
pickle.dump(rf, open('diabetes_model_rf.pkl', 'wb'))
pickle.dump(imputer, open('diabetes_imputer.pkl', 'wb'))
importancias.to_csv('importancia_variaveis.csv', index=False)
files.download('diabetes_model_rf.pkl')
files.download('diabetes_imputer.pkl')
files.download('importancia_variaveis.csv')
